# Enrichment analysis in search of peripheral metabollic subsystems

In [2]:
import pandas as pd
import numpy as np

In [ ]:
# Read datafile
data = pd.read_csv('periphericalRxns_iJN1480.tsv', sep = '\t', encoding = 'latin1')
data.head()

,Subsystem,Peripherical reactions,Not peripherical reactions,Total
0,Fatty Acids Biosynthesis,8,67,75
1,Alternate Carbon and Nitrogen Source,61,45,106
2,Lysine Metabolism,0,34,34
3,Cofactor and Prosthetic Group Biosynthesis,27,163,190
4,Fatty Acids Metabolism,132,233,365


In [4]:
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

enrich_analysis_results = []

subsystem_list = data["Subsystem"]

b = data.loc[data["Subsystem"] == 'Total', "Peripherical reactions"].iloc[0]
d = data.loc[data["Subsystem"] == 'Total', "Not peripherical reactions"].iloc[0]

for subsystem in subsystem_list:
    a = data.loc[data["Subsystem"] == subsystem, "Peripherical reactions"].iloc[0]
    c = data.loc[data["Subsystem"] == subsystem, "Not peripherical reactions"].iloc[0]

    cont_table = np.array([[a, b],[c,d]])  

    oddsratio, pvalue = fisher_exact(cont_table, alternative = "greater")

    enrich_analysis_results.append({
            "subsystem": subsystem,
            "peripherical": a,
            "no peripherical": c,
            "oddsratio": oddsratio,
            "pvalue": pvalue
        })

In [5]:
# Benjamini–Hochberg (FDR) correction
enrich_df = pd.DataFrame(enrich_analysis_results)
enrich_df["p_adj"] = multipletests(
    enrich_df["pvalue"],
    method = "fdr_bh"
)[1]

enrich_df

,subsystem,peripherical,no peripherical,oddsratio,pvalue,p_adj
0,Fatty Acids Biosynthesis,8,67,0.322919,9.998200e-01,1.000000e+00
1,Alternate Carbon and Nitrogen Source,61,45,3.666025,1.710703e-10,5.388329e-09
2,Lysine Metabolism,0,34,0.000000,1.000000e+00,1.000000e+00
3,Cofactor and Prosthetic Group Biosynthesis,27,163,0.447975,9.999816e-01,1.000000e+00
4,Fatty Acids Metabolism,132,233,1.532132,3.470336e-04,2.568049e-03
5,Glycolysis/Gluconeogenesis,0,29,0.000000,1.000000e+00,1.000000e+00
6,Aromatic Compounds Degradation,52,35,4.018032,4.892157e-10,6.033660e-09
7,"Valine, Leucine and Isoleucine Metabolism",1,32,0.084514,9.999659e-01,1.000000e+00
8,Urea Cycleamino Group Metabolism,0,7,0.000000,1.000000e+00,1.000000e+00
9,Alternate Carbon Source,55,38,3.914327,2.912610e-10,5.388329e-09


In [6]:
# Print only significant values
significant = enrich_df[enrich_df["p_adj"] <= 0.05]
significant

,subsystem,peripherical,no peripherical,oddsratio,pvalue,p_adj
1,Alternate Carbon and Nitrogen Source,61,45,3.666025,1.710703e-10,5.388329e-09
4,Fatty Acids Metabolism,132,233,1.532132,3.470336e-04,2.568049e-03
6,Aromatic Compounds Degradation,52,35,4.018032,4.892157e-10,6.033660e-09
9,Alternate Carbon Source,55,38,3.914327,2.912610e-10,5.388329e-09
32,Xenobiotic Tolerance,10,1,27.044444,1.852413e-05,1.713482e-04
33,Nitrogen Metabolism,4,0,inf,5.396643e-03,3.327930e-02


In [7]:
print(significant["subsystem"].tolist())

['Alternate Carbon and Nitrogen Source', 'Fatty Acids Metabolism', 'Aromatic Compounds Degradation', 'Alternate Carbon Source', 'Xenobiotic Tolerance', 'Nitrogen Metabolism']


These subsystems are associated with secondary metabolism